In [1]:
data_path = "/data/aholab/asr/corpus/Faktoria_Gaitu/asr/manifests/2025-01/2025-01_clean_norm.json"

In [2]:
from datasets import load_dataset
import torch

dataset = load_dataset("json", data_files=data_path, split="train")


/home/andoni.sudupe/envs/hubert_venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [29]:
import soundfile as sf

In [6]:
dataset['audio_filepath'][2]

'/data/aholab/asr/corpus/Faktoria_Gaitu/asr/wavs/2025-01/20230608_11200900_0015613406_002_002_FAKTORIA__20_p01/20230608_11200900_0015613406_002_002_FAKTORIA__20_p01_6.94_8.22.wav'

In [30]:
audio_input, sample_rate = sf.read(dataset['audio_filepath'][0])

In [31]:
audio_input

array([[0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ],
       ...,
       [0.00082397, 0.00079346],
       [0.00097656, 0.0010376 ],
       [0.00079346, 0.00085449]], shape=(13120, 2))

In [ ]:
from evaluate import load
from transformers import (
    Wav2Vec2CTCTokenizer,
    Wav2Vec2FeatureExtractor,
    Wav2Vec2Processor,
    HubertForCTC,
)

# Combine into processor
processor = Wav2Vec2Processor.from_pretrained('Ansu/mHubert-basque-ASR')

# Load model
model = HubertForCTC.from_pretrained('Ansu/mHubert-basque-ASR')

In [ ]:
from tqdm import tqdm
import librosa
import soundfile as sf
import torch 

# Prepare metrics
wer_metric = load("wer", trust_remote_code=True)
cer_metric = load("cer", trust_remote_code=True)

predictions = []
references = []

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
model.eval()

for i in tqdm(range(len(dataset))):
    wav_path = dataset[i]['audio_filepath']
    text = dataset[i]['text']
    # Read audio
    audio_input, sample_rate = sf.read(wav_path)
    # Convert to mono if stereo
    if len(audio_input.shape) == 2:
        audio_input = audio_input.mean(axis=1)
    # Resample if needed
    if sample_rate != processor.feature_extractor.sampling_rate:
        audio_input = librosa.resample(audio_input, orig_sr=sample_rate, target_sr=processor.feature_extractor.sampling_rate)
        sample_rate = processor.feature_extractor.sampling_rate
    # Preprocess
    inputs = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt")
    input_values = inputs.input_values.to(device)
    # Predict
    with torch.no_grad():
        logits = model(input_values).logits
    pred_ids = torch.argmax(logits, dim=-1)
    pred_str = processor.batch_decode(pred_ids)[0]
    predictions.append(pred_str)
    references.append(text)
    print(f'Predictions: {pred_str}')
    print(f'References: {text}')

# Calculate metrics
wer = wer_metric.compute(predictions=predictions, references=references)
cer = cer_metric.compute(predictions=predictions, references=references)

print(f"WER: {wer}")
print(f"CER: {cer}")

  0%|          | 1/24005 [00:11<75:38:26, 11.34s/it]

Predictions: eta hemen
References: eta hemen eh gaur egun


  0%|          | 2/24005 [00:18<61:06:43,  9.17s/it]

Predictions: ezgaur eko korporatiboa izateak berez ez daukat zertan eskuin muturreko jarrerak egon behar
References: sindikatu korporatibo izateak berez ez dauka zertan eh eskuin muturreko jarrerak eh egon behar


  0%|          | 3/24005 [00:22<43:59:34,  6.60s/it]

...

Predictions: eta ez ditugu ikusten gainera aztertu dugulako mahai gainean dagoena
References: eta ez ditugu ikusten gainera aztertu dugulako mahai gainean eh dagoena


  0%|          | 28/24005 [01:50<16:28:24,  2.47s/it]

Predictions: eta uste dugulako azterketa objektibo bat egiten baldin bada inongo baldintzarik gabe baldintzatuta egon gabe irakurketa horretan gure ustez esan daitekeena esan daitekeen bakarra da esaten ari garena
References: eta uste dugulako azterketa objektibo bat egiten baldin bada inongo eh baldintzarik gabe baldintzatuta egon gabe irakurketa horretan gure ustez esan daitekeena esan daitekeen bakarra da esaten ari garena


  0%|          | 28/24005 [01:52<26:45:58,  4.02s/it]


KeyboardInterrupt: 